<a href="https://colab.research.google.com/github/ferdonof/topicos-2/blob/master/calculadora_sueldos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Programa de cálculo de sueldos**
## 1 - Obtención de los datos

Los datos utilizados son los mismos que se utilizaron en el ejemplo de clase.

In [108]:
!git clone https://github.com/gabyaleperez/redesNeuronales.git

fatal: destination path 'redesNeuronales' already exists and is not an empty directory.


In [109]:
import numpy as np
import pandas as pd

filename = './redesNeuronales/datos_empleados_50000.csv'
data = pd.read_csv(filename, index_col=0)
print(data)


       sueldo_basico categoria  ausencias  cantidad_hijos  sueldo
0              33600         C          2               2   37600
1              14200         A          0               0   15570
2              10600         C          2               1   13850
3              16300         C          0               1   19620
4              20900         C          2               1   24150
...              ...       ...        ...             ...     ...
49995          22500         C          2               2   26500
49996          70600         A          1               0   71900
49997          23800         B          1               3   28050
49998          44800         A          1               3   48350
49999          57100         A          2               1   59150

[50000 rows x 5 columns]


## 2 - Procesamiento de los datos
Homogeneizar los datos, separarlos en  etiquetas para poder utilizalos de entrada.

In [110]:
from sklearn.model_selection import train_test_split

#Clasificación de los datos utilizando onehot
data = pd.get_dummies(data) 

#separamos el sueldo, para que no se use comoparte del entrenamiento
x = data.drop(['sueldo'], axis=1) 

y = np.array(data['sueldo'])

# utilizo la libreria para separar los datos de entrenamiento de los de testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2)

from pprint import pprint

# muestro los datos de entrenamiento (el 80% de los datos será para entrenar la red)
pprint(train_x)

       sueldo_basico  ausencias  cantidad_hijos  categoria_A  categoria_B  \
4596           25500          1               0            0            1   
23014          68700          2               2            0            1   
2484           53700          2               2            0            1   
14843          55600          1               0            0            0   
49340          63100          2               3            1            0   
...              ...        ...             ...          ...          ...   
37159          18000          1               1            0            1   
39978          50900          2               3            1            0   
22706          42100          1               0            0            0   
34852          79100          1               1            0            0   
27186          27700          1               2            1            0   

       categoria_C  
4596             0  
23014            0  
2484        

## 3 - Armado de la RN
Al tener 6 columnas, voy a necesitar 6 neuronas de entrada. Como la salida va a ser el sueldo (estimado) del empleado, voy a necesitar una neurona para la salida.

In [111]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.optimizers import Adam

model = Sequential()

# capa de entrada
model.add(Dense(50, input_shape = (6, ), activation='relu', kernel_initializer='uniform'))

# capas intermedia
# model.add(Dense(25, activation='relu', kernel_initializer='random_normal'))
# model.add(Dense(15, activation='relu', kernel_initializer='random_normal'))

# capa de salida
model.add(Dense(1, activation='relu'))


# hiperparámetros
loss_function = 'mean_squared_error' 
learning_rate = 0.01


model.compile(loss = loss_function, optimizer=Adam(learning_rate = learning_rate), metrics = ['accuracy'])

model.summary()


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_34 (Dense)            (None, 50)                350       
                                                                 
 dense_35 (Dense)            (None, 1)                 51        
                                                                 
Total params: 401
Trainable params: 401
Non-trainable params: 0
_________________________________________________________________


## 4 - Entrenamiento de la red

In [112]:
batch_size = 10000
epochs = 2000

model.fit(train_x, train_y, verbose=2, batch_size = batch_size, epochs = epochs)



Epoch 1/2000
4/4 - 0s - loss: 2777503744.0000 - 362ms/epoch - 90ms/step
Epoch 2/2000
4/4 - 0s - loss: 1878489472.0000 - 38ms/epoch - 9ms/step
Epoch 3/2000
4/4 - 0s - loss: 1022138368.0000 - 45ms/epoch - 11ms/step
Epoch 4/2000
4/4 - 0s - loss: 337881216.0000 - 28ms/epoch - 7ms/step
Epoch 5/2000
4/4 - 0s - loss: 27251126.0000 - 39ms/epoch - 10ms/step
Epoch 6/2000
4/4 - 0s - loss: 95697408.0000 - 35ms/epoch - 9ms/step
Epoch 7/2000
4/4 - 0s - loss: 188181536.0000 - 28ms/epoch - 7ms/step
Epoch 8/2000
4/4 - 0s - loss: 109664160.0000 - 32ms/epoch - 8ms/step
Epoch 9/2000
4/4 - 0s - loss: 18920878.0000 - 47ms/epoch - 12ms/step
Epoch 10/2000
4/4 - 0s - loss: 9101655.0000 - 40ms/epoch - 10ms/step
Epoch 11/2000
4/4 - 0s - loss: 29417358.0000 - 35ms/epoch - 9ms/step
Epoch 12/2000
4/4 - 0s - loss: 29070026.0000 - 30ms/epoch - 7ms/step
Epoch 13/2000
4/4 - 0s - loss: 12353043.0000 - 30ms/epoch - 7ms/step
Epoch 14/2000
4/4 - 0s - loss: 3206275.0000 - 38ms/epoch - 10ms/step
Epoch 15/2000
4/4 - 0s - loss

## 5 - Evaluar el resultado

In [113]:
result = model.evaluate(test_x, test_y)

313/313 [==============================] - 1s 2ms/step - loss: 102504.3438


## 6 - Predicción de los nuevos datos

In [114]:
pred_y = model.predict(test_x)
  
print("Resultados de las predicciones\n")  
print("|\tEsperado $\t|\tObtenido $|\t Desvio\t|")
print("-------------------------------------------------")

for i in range(3):
  desvio = 0
  if(pred_y[i][0] != 0):
    desvio = test_y[i] / pred_y[i][0]
  print("|\t",test_y[i],"\t\t|\t",pred_y[i][0],"|\t", desvio,"\t|")


313/313 [==============================] - 1s 1ms/step
Resultados de las predicciones

|	Esperado $	|	Obtenido $|	 Desvio	|
-------------------------------------------------
|	 66000 		|	 66148.65 |	 0.9977528121736237 	|
|	 38600 		|	 38756.516 |	 0.995961566139887 	|
|	 31800 		|	 31783.807 |	 1.0005094845799338 	|
